<a href="https://colab.research.google.com/github/itachiSenpai17/WikiPyApp/blob/main/run_app_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install wikipedia wikipedia-api plotly

# Download the app.py from your GitHub repo
!wget https://raw.githubusercontent.com/itachiSenpai17/WikiPyApp/main/app.py

# Run the script
!python3 app.py